# <center>CryptoCurrencies CA#1<center>
<h1><center>Introduction to Cryptography<center></h1>

#### <center>"Name - Family name"<center>

## StoryLine:
We have TinTin and Haddock on two seperate islands and they want to send messages to each other using morse codes!!    

However, there is a problem over there. Roberto Rastapopoulos (Their enemy!) is listening to their conversation to figure out what they are saying to each other!    

Now TinTin wants to use some encryption and decryption to ensure that Roberto couldn't understand what they are saying. Help TinTin to acheive his goal!

## Part 1: Symmetric Cryptography



**Important Note**: In all steps, use **AES-128** encryption method for encryption/decryption. You can use python cryptography libraries for this purpose.

TinTin decided to design a symmetirc encryption scheme for his connection with haddock. He asked you to design it for him using these steps:

In [54]:
import os
import string
import random
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa, padding

In [2]:
def symmetric_encryptor(input_message: str, key: str) -> bytes:
    fernet_key = Fernet(key)

    cipher_text = fernet_key.encrypt(input_message.encode())

    return cipher_text

In [16]:
def symmetric_decryptor(input_ciphertext: str, key: bytes) -> str:
    fernet_key = Fernet(key)

    decrypted_message = fernet_key.decrypt(input_ciphertext.encode()).decode()

    return decrypted_message

In [28]:
def symmetric_verifier(plaintext:str, ciphertext:str, key:str):
    if plaintext == ciphertext:
        return True
    else:
        return False

Now test your functions by encrypting & decrypting the ((confidential-message.txt)) file.

In [5]:
random_key = Fernet.generate_key()

with open("Confidential-Message.txt", "r") as file:
    confidential_message = file.read()

encrypted_message = symmetric_encryptor(confidential_message, random_key)

with open("Encrypted-Message.txt", "wb") as file:
    file.write(encrypted_message)

print("Confidential message encrypted and saved as Encrypted-Message.txt")

Confidential message encrypted and saved as Encrypted-Message.txt


In [19]:
with open("Encrypted-Message.txt", "r") as file:
    Encrypted_message = file.read()

decrypted_message = symmetric_decryptor(Encrypted_message, random_key)

with open("Decrypted-Message.txt", "w") as file:
    file.write(decrypted_message)

print("Confidential message encrypted and saved as Decrypted-Message.txt")

Confidential message encrypted and saved as Decrypted-Message.txt


In [29]:
if symmetric_verifier(confidential_message, decrypted_message, random_key):
    print("The original message and the decrypted message are the same.")
else:
    print("The original message and the decrypted message are different.")

The original message and the decrypted message are the same.


## Part 2: Asymmetric Cryptography

After TinTin found how to use symmetric encryption using a key, he faced another problem! How to share a key with haddock without letting Roberto finds it?        

Therefore, he decides to use an asymmetric crypto system.    
help him to acheive his goal by writing a program which:

In [32]:
def RSA_key_generator():

    private_key = rsa.generate_private_key(
        public_exponent=65537,  # Commonly used value for the public exponent
        key_size=2048,          
        backend=default_backend()
    )

    public_key = private_key.public_key()

    private_key_pem = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    )
    public_key_pem = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )

    return private_key_pem, public_key_pem    

In [83]:
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.hashes import SHA256
from cryptography.hazmat.backends import default_backend

In [77]:
def asymmetric_encryptor(public_key_str: str, input_message: str) -> bytes:
    # Deserialize the public key from PEM format
    public_key = serialization.load_pem_public_key(public_key_str.encode(), backend=default_backend())

    # Encrypt the input message using the public key
    encrypted_message = public_key.encrypt(
        input_message.encode(),
        padding.OAEP(
            mgf=padding.MGF1(algorithm=serialization.Algorithm.SHA256),
            algorithm=serialization.Algorithm.SHA256,
            label=None
        )
    )

    return encrypted_message

In [78]:
def asymmetric_decryptor(private_key_str: str, ciphertext: bytes) -> str:
    # Deserialize the private key from PEM format
    private_key = serialization.load_pem_private_key(private_key_str.encode(), password=None, backend=default_backend())

    # Decrypt the ciphertext using the private key
    decrypted_message = private_key.decrypt(
        ciphertext,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=serialization.Algorithm.SHA256),
            algorithm=serialization.Algorithm.SHA256,
            label=None
        )
    ).decode()

    return decrypted_message

In [79]:
def symmetric_verifier(plaintext:str, ciphertext:str, public_key:str):
    if plaintext == ciphertext:
        return True
    else:
        return False

Now test your functions by encrypting & decrypting the ((confidential-message.txt)) file again using new asummetric functions.

In [80]:
private_key, public_key = RSA_key_generator()

# Save the keys to files
with open("private_key.pem", "wb") as private_key_file:
    private_key_file.write(private_key)

with open("public_key.pem", "wb") as public_key_file:
    public_key_file.write(public_key)

print("RSA key pair generated and saved as private_key.pem and public_key.pem")


RSA key pair generated and saved as private_key.pem and public_key.pem


In [82]:
with open("public_key.pem", "r") as public_key_file:
    # public_key_str = public_key_file.read()
    public_key = public_key_file.read()

with open("private_key.pem", "r") as private_key_file:
    private_key = private_key_file.read()

# Encrypt the confidential message using the public key
locked_message = asymmetric_encryptor(public_key_str, confidential_message)
unlcked_message = symmetric_decryptor(private_key, locked_message)


AttributeError: module 'cryptography.hazmat.primitives.serialization' has no attribute 'Algorithm'

In [73]:
unlocked_message = 'jnjknkniun'
if symmetric_verifier(confidential_message, unlocked_message, public_key_str):
    print("The original message and the decrypted message are the same.")
else:
    print("The original message and the decrypted message are different.")

The original message and the decrypted message are different.


## Part 3: Digital signature

Write a function to sign a given message in the text file (Confidentail message) using private key which was made in the part 2.   
You have to sign the hash of the message, not the whole message (why?). (Use SHA-256 for hashing)

In [ ]:
def sign_message(private_key:str, input_message: str):
    # Write a code to sign a message using your private key.
    # CODE HERE:
    
    
    
    return message_signature

In [ ]:
def message_verification(input_message:str, signature:str, public_key:str):
    # Write a code to verify the signature of the input message
    # output should be boolean(ture or false)
